wsl (vpnkit as a service)

# Week 48 - December 23

## Friday 12/01

[wsl-vpnkit](https://github.com/sakai135/wsl-vpnkit/releases) v0.4 does not support running wsl-vpnkit using `wsl.exe -d wsl-vpnkit service wsl-vpnkit start`. A systemd service file is provided to help us setup a systemd service in distro to run wsl-vpnkit in the background.

Here are the steps:

**dependencies**
```bash
sudo apt install -y iproute2 iptables iputils-ping dnsutils wget
```

**binary installation**
```bash
VERSION=v0.4.1
sudo mkdir -p /opt/vpnkit

cd /opt/vpnkit
sudo wget https://github.com/sakai135/wsl-vpnkit/releases/download/$VERSION/wsl-vpnkit.tar.gz
sudo tar --strip-components=1 -xzf wsl-vpnkit.tar.gz \
  app/wsl-vpnkit \
  app/wsl-gvproxy.exe \
  app/wsl-vm \
  app/wsl-vpnkit.service \
  -C /opt/vpnkit
sudo rm wsl-vpnkit.tar.gz

cd -
```

**configuration**
```bash
sudo cat <<'EOT' > /opt/vpnkit/wsl-vpnkit.service
[Unit]
Description=wsl-vpnkit

[Service]
# for wsl-vpnkit setup as a standalone script
# important to set type to idle, we want the service to be one of the last ones to be executed
Type=idle
# before running the wsl-vpnkit script we want to provide the correct WSL_INTEROP variable
ExecStart=/bin/sh -c '. /opt/vpnkit/wsl-interop-env.sh; /opt/vpnkit/wsl-vpnkit'
Environment=VMEXEC_PATH=/opt/vpnkit/wsl-vm
Environment=GVPROXY_PATH=/opt/vpnkit/wsl-gvproxy.exe

Restart=always
KillMode=mixed

[Install]
WantedBy=multi-user.target
EOT


sudo cat <<'EOT' > /opt/vpnkit/wsl-interop-env.sh
#!/bin/sh

export WSL_INTEROP=
for socket in $(ls /run/WSL|sort -n); do
if ss -elx | grep "$socket"; then
  export WSL_INTEROP=/run/WSL/$socket
else
  rm $socket
fi
done
EOT

sudo chmod +x /opt/vpnkit/wsl-interop-env.sh
sudo cp /opt/vpnkit/wsl-vpnkit.service /etc/systemd/system/
sudo systemctl enable wsl-vpnkit.service
```

**start service**
```bash
sudo systemctl start wsl-vpnkit.service
```

and to test

from powershell
```shell
wsl -t wsl-vpnkit
```

from wsl image
```bash
host google.fr
```

we can see `wsl-vpnkit` logs with `sudo systemctl status wsl-vpnkit.service`

and as soon as it is ok, we can unregister wsl-vpnkit

```shell
wsl --unregister wsl-vpnkit
```